In [1]:
from collections import Counter
from collections import defaultdict
from random import random
from collections import defaultdict
from IPython import display
from PIL import Image
from torch.autograd import Variable
import numpy as np
import torch
import torch.nn.functional as F 
from torchvision import models
import json
import itertools
import torch
import torch.nn as nn
import torch.optim as optim 
import matplotlib.ticker as ticker 
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
!pip3 install opencv-python

In [4]:
import numpy as np
import pandas as pt
import matplotlib as plt
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
benign_train = np.array(glob("C:/Users/garim/Documents/federatedLearning/data/train/benign/*"))
malignant_train = np.array(glob("C:/Users/garim/Documents/federatedLearning/data/train/malignant/*"))
print('There are %d total benign cancer images.' % len(benign_train))
print('There are %d total malignant cancer images.' % len(malignant_train))

There are 467 total benign cancer images.
There are 3575 total malignant cancer images.


In [5]:
device =''
if torch.cuda.is_available() :
  device = torch.device("cuda")
  print("cuda") 
else:
  device = torch.device("cpu")

cuda


In [6]:
# concat the data and make the train and validation images
combined_data_train = np.concatenate((benign_train, malignant_train), axis =0)
print (combined_data_train.shape)

(4042,)


In [7]:
#generating the labels
benign_train_label = np.zeros(len(benign_train))
malignant_train_label = np.ones(len(malignant_train)) # we are using 1 for malignant and 0 for benign cancer
combined_labels_train = np.concatenate((benign_train_label, malignant_train_label), axis = 0)
print (combined_labels_train.shape)

(4042,)


In [8]:
print (combined_data_train[0]) # np array of path files to images

C:/Users/garim/Documents/federatedLearning/data/train/benign\SOB_B_A-14-22549AB-200-016.png


In [9]:
# shuffle the images in the training np array to avoid overfitting
random_array = np.arange(combined_data_train.shape[0])
np.random.shuffle(random_array)
X_train = combined_data_train[random_array]
Y_train = combined_labels_train[random_array]
print (X_train.shape)
print (Y_train.shape)
'''
for i in range (20):
    print (X_train[i],"--->",Y_train[i],"--->", Y_train_encoded[i])
#print (X_train[:20],"--->", Y_train[:20])
'''

(4042,)
(4042,)


'\nfor i in range (20):\n    print (X_train[i],"--->",Y_train[i],"--->", Y_train_encoded[i])\n#print (X_train[:20],"--->", Y_train[:20])\n'

In [10]:
# convert to one hot encoding
Y_train_encoded = np.zeros([combined_data_train.shape[0], 2])
print (Y_train_encoded.shape)
for i in range(0, len(Y_train)):
  if combined_labels_train[i] == 0: #benign  one hot encoding = 1,0
    Y_train_encoded[i][0] = 1
  else: # malignant one hot encoding = 0,1
    Y_train_encoded[i][1] = 1

(4042, 2)


In [11]:
# Define a global transformer to appropriately scale images and subsequently convert them to a Tensor.
img_size = 224
loader = transforms.Compose([
  transforms.Resize(img_size),
  transforms.CenterCrop(img_size),
  transforms.ToTensor(),
]) 
def load_image(filename):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    image = Image.open(filename).convert('RGB')
    image_tensor = loader(image).float()
    image_var = Variable(image_tensor).unsqueeze(0)
    #return image_var.cuda()
    #print (image_var.shape)
    return image_var
trial = load_image(combined_data_train[0])
print (type(trial))

<class 'torch.Tensor'>


In [12]:
# load a pretrained model
# modify the pretrained model
# vectorize all images by passing through vgg16
# visualize
# Write custom dataset
# write custom neural net
# train

In [13]:
# load a pretrained model
vgg_model = models.vgg16(pretrained = True)
vgg_model.eval()
vgg_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
vgg_model = vgg_model.to(device)

In [15]:
# modify the pretrained model - basically generate the feature map by removing the last layer
# saving original number of in features and out features for the last layer
original_infeatures = vgg_model.classifier[6].in_features
original_outfeatures = vgg_model.classifier[6].out_features
#print (original_infeatures)
# modifying the model
vgg_model.classifier = nn.Sequential(*[vgg_model.classifier[i] for i in range(5)])
vgg_model.eval() # remove the dropout for now


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
# vectorize all images by passing through vgg16
#training_vectors = torch.zeros((X_train.shape[0], 4096), device=device)
training_vectors = np.zeros((X_train.shape[0], 4096))
for i in range(0, X_train.shape[0]):
  img = load_image(X_train[i]).to(device)
  #img = load_image(X_train[i])
  features = vgg_model(img)
  training_vectors[i] = features.cpu().detach().numpy()
  #print (training_vectors[i].device)
  #training_vectors[i] = features.cpu().detach().numpy()
  if i%100 ==0:
    print ("Number of images vectorized = ", i)

Number of images vectorized =  0
Number of images vectorized =  100
Number of images vectorized =  200
Number of images vectorized =  300
Number of images vectorized =  400
Number of images vectorized =  500
Number of images vectorized =  600
Number of images vectorized =  700
Number of images vectorized =  800
Number of images vectorized =  900
Number of images vectorized =  1000
Number of images vectorized =  1100
Number of images vectorized =  1200
Number of images vectorized =  1300
Number of images vectorized =  1400
Number of images vectorized =  1500
Number of images vectorized =  1600
Number of images vectorized =  1700
Number of images vectorized =  1800
Number of images vectorized =  1900
Number of images vectorized =  2000
Number of images vectorized =  2100
Number of images vectorized =  2200
Number of images vectorized =  2300
Number of images vectorized =  2400
Number of images vectorized =  2500
Number of images vectorized =  2600
Number of images vectorized =  2700
Numb

In [17]:
training_vectors.shape

(4042, 4096)

In [18]:
# visualize
# Write custom dataset
# write custom neural net
# train

In [19]:
# visualize

In [20]:
# creat image dataset 
class ImageDataset(Dataset):
  def __init__(self,X,Y):
    self.n_samples = X.shape[0]
    self.X = torch.from_numpy(X)
    self.Y = torch.from_numpy(Y)
  def __getitem__(self,index):
    return self.X[index], self.Y[index]
    
  def __len__(self):
    return self.n_samples

In [21]:
# write custom neural net
class Imageclassification(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
        super(Imageclassification, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.l2 = nn.Linear(hidden_size, num_classes)
  def forward(self, x):
        out = self.l1(x)
        out = self.sigmoid(out)
        out = self.l2(out)
        out = self.sigmoid(out) # We require this if we use BCE LOSS
        return out

In [22]:
# set variables
n_inputs = 4096
n_hidden = 512
n_outputs = 2
model = Imageclassification(n_inputs, n_hidden, n_outputs)
learning_rate=0.01
# loss and optimizer
#criterion = nn.MultiLabelSoftMarginLoss()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate )

In [23]:
dataset = ImageDataset(training_vectors,Y_train_encoded)
dataloader_train = DataLoader(dataset = dataset, batch_size = 50, shuffle = True)
dataiter = iter(dataloader_train)

In [24]:
model = model.double()
outputs = model(dataset[0][0])

In [25]:
# train
total_loss = []
def train(model, learning_rate=0.01, batch_size=50, epochs=100):
    """
    Training function which takes as input a model, a learning rate and a batch size.
  
    After completing a full pass over the data, the function exists, and the input model will be trained.
    """
    # -- Your code goes here --
    for i in range(0,epochs):
        model.train()
        for b_index, (featueres, labels) in enumerate(dataloader_train):
            #print (featueres.shape)
            #print (b_index)
            outputs = model(featueres)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
            if(b_index)%50 == 0:
                print (f'epoch {i+1}/{epochs} Batch {b_index+1} loss {loss}')
                total_loss.append(loss)

In [26]:
train(model)

epoch 1/100 Batch 1 loss 0.6178126700687341
epoch 1/100 Batch 51 loss 0.49449457545441744
epoch 2/100 Batch 1 loss 0.40621323052398806
epoch 2/100 Batch 51 loss 0.36777174338603474
epoch 3/100 Batch 1 loss 0.3286144106946405
epoch 3/100 Batch 51 loss 0.4059856129450188
epoch 4/100 Batch 1 loss 0.19585818802788338
epoch 4/100 Batch 51 loss 0.36733348967696705
epoch 5/100 Batch 1 loss 0.28505020640784723
epoch 5/100 Batch 51 loss 0.285881886055258
epoch 6/100 Batch 1 loss 0.40599161836354086
epoch 6/100 Batch 51 loss 0.3659117422771388
epoch 7/100 Batch 1 loss 0.28241338015106376
epoch 7/100 Batch 51 loss 0.44263853033190287
epoch 8/100 Batch 1 loss 0.4088440876735588
epoch 8/100 Batch 51 loss 0.20813221046005836
epoch 9/100 Batch 1 loss 0.2861567617677198
epoch 9/100 Batch 51 loss 0.32810142608067366
epoch 10/100 Batch 1 loss 0.24337533602566805
epoch 10/100 Batch 51 loss 0.4044005933995203
epoch 11/100 Batch 1 loss 0.2822782727050409
epoch 11/100 Batch 51 loss 0.4080559019177734
epoch 

In [ ]:
# Performance

In [ ]:
# testing

In [ ]:
# visualize